In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import remap_values as rv
import good_plots   as gp

import cPickle

In [317]:
reload( rv )

<module 'remap_values' from 'remap_values.py'>

In [318]:
train_df = pd.read_csv('inp_data/train.csv')
test_df  = pd.read_csv('inp_data/test.csv')

train_df, test_df = rv.run_tan( train_df, test_df )

In [291]:
big_df = train_df.drop( 'SalePrice', axis=1).append( test_df, ignore_index=True)

In [41]:
#PCA?
#Preprocess?

In [319]:
corr_series = num_df.corrwith( num_df['SalePrice'] ).sort_values( ascending=False )
print corr_series

SalePrice           1.000000
OverallQual         0.800858
AreaInside          0.729888
GrLivArea           0.720516
NeighBin            0.696339
ExterQual           0.694628
KitchenQual         0.666217
GarageCars          0.649256
TotalBsmtSF         0.646584
GarageArea          0.636964
NBath               0.635939
1stFlrSF            0.625235
BsmtQual            0.592073
TotRmsAbvGrd        0.537462
YearBuilt           0.535279
FireplaceQu         0.528841
YearRemodAdd        0.521428
GarageYrBlt         0.520632
MasVnrArea          0.473650
Fireplaces          0.466765
HeatingQC           0.435081
HeatingScale        0.435081
HasOpenPorchSF      0.419718
GarageFinish        0.419276
BsmtFinSF1          0.395923
BsmtExposure        0.373158
HasMasVnrArea       0.372950
HasMasVnr           0.372950
PartialPlan         0.371494
RecentRemodel       0.354686
                      ...   
Functional          0.109250
HasScreenPorch      0.093352
MoSold              0.056797
Has3SsnPorch  

In [320]:
col_list   = train_df.columns.values
large_list = []
for col in col_list:
    if ( train_df[col].max() > 10 ):
        print col, train_df[col].max()

Id 2919
MSSubClass 190
LotFrontage 200.0
LotArea 56600
YearBuilt 2010
YearRemodAdd 2010
MasVnrArea 1290.0
BsmtFinSF1 4010.0
BsmtFinSF2 1526.0
BsmtUnfSF 2140.0
TotalBsmtSF 5095.0
1stFlrSF 5095
2ndFlrSF 1862
LowQualFinSF 1064
GrLivArea 5095
TotRmsAbvGrd 15
GarageYrBlt 2207.0
GarageArea 1488.0
WoodDeckSF 1424
OpenPorchSF 742
EnclosedPorch 1012
3SsnPorch 360
ScreenPorch 576
PoolArea 800
MiscVal 17000
MoSold 12
YrSold 2010
AreaInside 5095
Age 131
YearSinceRemodel 60
SalePrice 625000


In [305]:
invest = 'Neighborhood'

In [306]:
big_df[invest].unique()

array(['NAmes', 'Gilbert', 'StoneBr', 'BrDale', 'NPkVill', 'NridgHt',
       'Blmngtn', 'NoRidge', 'Somerst', 'SawyerW', 'Sawyer', 'NWAmes',
       'OldTown', 'BrkSide', 'ClearCr', 'SWISU', 'Edwards', 'CollgCr',
       'Crawfor', 'Blueste', 'IDOTRR', 'Mitchel', 'Timber', 'MeadowV',
       'Veenker'], dtype=object)

In [308]:
gp.plot_avg( train_df, invest, 'SalePrice' )

In [321]:
gp.hist_plot( big_df, invest )

In [264]:
gp.corr_plot( train_df.ix[:200,['SalePrice','LotFrontage','LotArea','BsmtFinSF1','BsmtFinSF2','OpenPorchSF','EnclosedPorch']], focus='SalePrice')